## ewf-wfp-02-01-03 - Long Term Averages of Land Surface Temperature Time Series

Long Term Averages of Land Surface Temperature Time Series

---

### <a name="service">Service definition

In [ ]:
service = dict([('title', 'Long Term Averages of Land Surface Temperature Time Series'),
                ('abstract', 'Long Term Averages of Land Surface Temperature Time Series'),
                ('id', 'ewf-wfp-02-01-03')])

### <a name="parameter">Parameter Definition 

In [ ]:
N_1 = dict([('id', 'N_1'),
                          ('value', 'False'),
                          ('title', 'No Aggregation'),
                          ('abstract', 'No aggregation')])

In [ ]:
N_3 = dict([('id', 'N_3'),
                          ('value', 'True'),
                          ('title', '30 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [ ]:
N_6 = dict([('id', 'N_6'),
                          ('value', 'False'),
                          ('title', '60 Day Aggregation'),
                          ('abstract', 'Get a 30 day aggregation')])

In [ ]:
 N_9 = dict([('id', 'N_9'),
                          ('value', 'False'),
                          ('title', '90 Day Aggregation'),
                          ('abstract', 'Get a 90 day aggregation')])

In [ ]:
 N_12 = dict([('id', 'N_12'),
                          ('value', 'False'),
                          ('title', '120 Day Aggregation'),
                          ('abstract', 'Get a 120 day aggregation')])

In [ ]:
 N_15 = dict([('id', 'N_15'),
                          ('value', 'False'),
                          ('title', '150 Day Aggregation'),
                          ('abstract', 'Get a 150 day aggregation')])

In [ ]:
 N_18 = dict([('id', 'N_18'),
                          ('value', 'False'),
                          ('title', '180 Day Aggregation'),
                          ('abstract', 'Get a 180 day aggregation')])

In [ ]:
 N_27 = dict([('id', 'N_27'),
                          ('value', 'False'),
                          ('title', '270 Day Aggregation'),
                          ('abstract', 'Get a 270 day aggregation')])

In [ ]:
 N_36 = dict([('id', 'N_36'),
                          ('value', 'False'),
                          ('title', '360 Day Aggregation'),
                          ('abstract', 'Get a 360 day aggregation')])

In [ ]:
regionOfInterest = dict([('id', 'regionOfInterest'),
                         ('value', 'POLYGON((11.5030755518998 -11.1141633706909,41.0343255518998 -11.1141633706909,41.0343255518998 -34.9763656693858,11.5030755518998 -34.9763656693858,11.5030755518998 -11.1141633706909))'),
                         ('title', 'WKT Polygon for the Region of Interest'),
                         ('abstract', 'Set the value of WKT Polygon')])

In [ ]:
nameOfRegion = dict([('id', 'nameOfRegion'),
                     ('value', 'SouthernAfrica'),
                     ('title', 'Name of Region'),
                     ('abstract', 'Name of the region of interest'),
                     ('minOccurs', '1')])

In [ ]:
indexAggCat = dict([('id', 'indexAggCat'),
             ('value', 'better-wfp-02-01-02'),
             ('title', 'indexAggCat'),
             ('abstract', 'index to access catalog of aggregated land surface temperature time series'),
             ('minOccurs', '1')])

In [ ]:
apikeyAggCat = dict([('id', 'apikeyAggCat'),
                ('value', ''),
                ('title', 'apikeyAggCat'),
                ('abstract', 'apikey to access indexAggCat catalog'),
                ('minOccurs', '1')])

### <a name="runtime">Runtime parameter definition

**Input identifiers**

This is the MDOIS stack of products' identifiers

In [ ]:
input_identifiers = ('0009d1c7-e754-4b77-8811-7de9d0436d5c/LST_SouthernAfrica_N3_averages_2015-01-05_2015-01-25.tif',
                     '36bc25f2-1b40-4514-8bf0-f6b6883f499a/LST_SouthernAfrica_N3_averages_2016-01-05_2016-01-25.tif',
                     '032cf334-c40a-4c24-b504-169f37dc562f/LST_SouthernAfrica_N3_averages_2017-01-05_2017-01-25.tif')

**Input references**

This is the MODIS stack catalogue references

In [ ]:
input_references =  ('http://sb-10-150-0-22.better.terradue.int:80/sbws/wps/ewf-wfp-02-01-02/0000019-190801000013916-oozie-oozi-W/results/search?id=0009d1c7-e754-4b77-8811-7de9d0436d5c/LST_SouthernAfrica_N3_averages_2015-01-05_2015-01-25.tif',
                     'http://sb-10-150-0-22.better.terradue.int:80/sbws/wps/ewf-wfp-02-01-02/0000020-190801000013916-oozie-oozi-W/results/search?id=36bc25f2-1b40-4514-8bf0-f6b6883f499a/LST_SouthernAfrica_N3_averages_2016-01-05_2016-01-25.tif',
                     'http://sb-10-150-0-22.better.terradue.int:80/sbws/wps/ewf-wfp-02-01-02/0000021-190801000013916-oozie-oozi-W/results/search?id=032cf334-c40a-4c24-b504-169f37dc562f/LST_SouthernAfrica_N3_averages_2017-01-05_2017-01-25.tif')

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = "/workspace/modis/outputs/output02"

**Aux folders**

In [ ]:
output_folder = ''

In [ ]:
temp_folder = 'temp'

#### Import Modules

In [ ]:
import os
import shutil

import cioppy

import sys
import string
import numpy as np
from osgeo import gdal, ogr, osr
from shapely.wkt import loads

import datetime as dt

import pandas as pd
import geopandas as gpd

import pdb

ciop = cioppy.Cioppy()

#### Auxiliary vars

In [ ]:
check_results = False

#### Auxiliary methods

In [ ]:
def rm_cfolder(folder):
    
    for the_file in os.listdir(folder):
        file_path = os.path.join(folder, the_file)
        try:
            if os.path.isfile(file_path):
                os.unlink(file_path)
            elif os.path.isdir(file_path): shutil.rmtree(file_path)
        except Exception as e:
            print(e)
    

# get metadata from catalog
def get_input_metadata (input_refs):
    
    # for each product get metadata
    Result_Prod = []
    
    for index,product_ref in enumerate(input_refs):
        
        # since the search is by identifier 
        Result_Prod.append(ciop.search(end_point = product_ref,params =[],output_fields='self,identifier,startdate,enclosure,title,startdate,enddate,wkt',creds='{}:{}'.format(indexAggCat['value'],apikeyAggCat['value']))[0] )
    

    input_metadata = gpd.GeoDataFrame.from_dict(Result_Prod)

    input_metadata['startdate'] = pd.to_datetime(input_metadata['startdate'])
    input_metadata['enddate'] = pd.to_datetime(input_metadata['enddate'])
    
    return input_metadata


    
def get_metadata(filepath):
    ds = gdal.Open(filepath)
    projection = ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    no_data_value = ds.GetRasterBand(1).GetNoDataValue()
    data_type = ds.GetRasterBand(1).DataType
    return projection, geotransform, no_data_value, data_type


def matrix_sum(mat1, mat2, no_data_value=None):
    if no_data_value is not None:
        if not isinstance(mat1, int):
            mat1[(mat1 == no_data_value)] = 0
        if not isinstance(mat2, int):
            mat2[(mat2 == no_data_value)] = 0
            
            
    msum = mat1 + mat2
        
    msum[(mat1 == 0)] = 0
    msum[(mat2 == 0)] = 0
        
    return msum


def write_output_image(filepath, output_matrix, image_format, data_format, output_projection=None, output_geotransform=None, mask=None, no_data_value=None):
    
    driver = gdal.GetDriverByName(image_format)
    out_rows = np.size(output_matrix, 0)
    out_columns = np.size(output_matrix, 1)
    if mask is not None:
        output = driver.Create(filepath, out_columns, out_rows, 2, data_format)
        mask_band = output.GetRasterBand(2)
        mask_band.WriteArray(mask)
    else:
        output = driver.Create(filepath, out_columns, out_rows, 1, data_format)
        
    if output_projection is not None:
        output.SetProjection(output_projection)
    if output_geotransform is not None:
        output.SetGeoTransform(output_geotransform)
    
    raster_band = output.GetRasterBand(1)
    if no_data_value is not None:
        raster_band.SetNoDataValue(no_data_value)
    raster_band.WriteArray(output_matrix)
    gdal.Warp(filepath, output, format="GTiff", outputBoundsSRS='EPSG:4326', xRes=output_geotransform[1], yRes=-output_geotransform[5], targetAlignedPixels=True)
    
    output.FlushCache()
    
def calc_average(matrix_list, n_years):
    if not isinstance(matrix_list, list):
        return 0
    result = matrix_list[0]
    for i in range(1, n_years):
        result = matrix_sum(result, matrix_list[i])
    
    return np.divide(result, (n_years*1.00))

def get_matrix_list(image_list):
    mat_list = []
    for img in image_list:
        dataset = gdal.Open(img)
        product_array = dataset.GetRasterBand(1).ReadAsArray()
        mat_list.append(product_array)
        dataset = None
    return mat_list


def calc_lta(file_list):
    
    if file_list:
        
        n_years = len(file_list)
        agr_period_matrix = get_matrix_list(file_list)
        print('Aggregations converted to matrices')
        lta = calc_average(agr_period_matrix, n_years)
        projection, geotransform, no_data_value, data_type = get_metadata(file_list[0])
        
        return lta, projection, geotransform, no_data_value, data_type
    
    else:
        return None, None, None


def write_output(temp_folder, lta, period_start_date, period_end_date, startLTAyear, endLTAyear, product_type, period_N, agr_type, region, projection, geo_transform, image_format, no_data_value, data_type):
    
    start_day_month = str(period_start_date.month) + '-' + str(period_start_date.day)
    end_day_month = str(period_end_date.month) + '-' + str(period_end_date.day)
  
    output_name = os.path.join(temp_folder, 'LTA_' + product_type + '_' + region + '_' + str(period_N) + '_' + agr_type + '_' + start_day_month + '_' + end_day_month + '_' + str(startLTAyear) + '_' + str(endLTAyear) + '.tif')
    
    write_output_image(output_name, lta, image_format, data_type, projection, geo_transform, no_data_value=no_data_value)
    
    return output_name


def get_formatted_date(date_str):
    date = dt.datetime.strftime(date_str, '%Y-%m-%dT00:00:00Z')
    return date


def write_properties_file(output_name, first_date, last_date, region_of_interest):
    
    title = 'Output %s' % output_name
    
    first_date = get_formatted_date(first_date)
    last_date = get_formatted_date(last_date)
    
    with open(output_name + '.properties', 'wb') as file:
        file.write('title=%s\n' % title)
        file.write('date=%s/%s\n' % (first_date, last_date))
        file.write('geometry=%s' % (region_of_interest))

#### Auxiliary folders

In [ ]:
#Create folders
#if not os.path.isdir(data_path):
#    os.mkdir(data_path)

if len(output_folder) > 0:
    if not os.path.isdir(output_folder):
        os.mkdir(output_folder)

if not os.path.isdir(temp_folder):
    os.mkdir(temp_folder)

#### Workflow

Compute Long Term Averages

In [ ]:
message = 'Getting metadata' 
ciop.log('INFO', message)

input_metada = get_input_metadata(input_references)


message = 'computing LTA' 
ciop.log('INFO', message)

# N time steps
nlist = [N_1['value'], N_3['value'], N_6['value'], N_9['value'], N_12['value'], N_15['value'], N_18['value'], N_27['value'], N_36['value']]
nlist = [n == 'True' for n in nlist]
nvalues = [1, 3, 6, 9, 12, 15, 18, 27, 36]

file_list = []
for bl,nv in zip(nlist, nvalues):
    
    # only works for selected N time steps
    if bl:
        
        N = nv
        
        file_list = [os.path.join(data_path, os.path.basename(enclosure).split('?')[0]) for enclosure in input_metada['enclosure']]
        
        lta, projection, geotransform, no_data_value, data_type = calc_lta(file_list)


In [ ]:
# CHECK
if check_results:
    
    import matplotlib
    import matplotlib.pyplot as plt

    fig = plt.figure()
    plt.imshow(lta)
    plt.show()

#### write output

In [ ]:
message = 'Writing output' 
ciop.log('INFO', message)


# get start and last date from metadata
startdate = input_metada['startdate'].min().strftime('%Y-%m-%d')
enddate = input_metada['enddate'].max().strftime('%Y-%m-%d')

startLTAyear = int(input_metada['startdate'].min().strftime('%Y'))
endLTAyear = int(input_metada['startdate'].max().strftime('%Y'))

print(startLTAyear)
print(endLTAyear)


filename = os.path.splitext(os.path.basename(file_list[0]))[0].split('_')

startdate = dt.datetime.strptime(startdate, '%Y-%m-%d')
enddate = dt.datetime.strptime(enddate, '%Y-%m-%d')


agr = filename[3]
prod_type = filename[0]
N_value = filename[2]
region = filename[1]


if lta is not None:

    filename = write_output(output_folder, lta, startdate, enddate, startLTAyear, endLTAyear, prod_type, N_value, agr, region, projection, geotransform, 'GTiff', no_data_value, data_type)
    print(filename)
    write_properties_file(filename, startdate, enddate, regionOfInterest['value'])



#### Remove temporay files and folders

In [ ]:
message = 'Removing temporary files' 
ciop.log('INFO', message)

rm_cfolder(temp_folder)

os.rmdir(temp_folder)